# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.9963092439264216),
 (1, 1.9963092439264216),
 (2, 1.9963092439264216),
 (3, 1.9963092439264216),
 (4, 1.9963092439264216)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('is', 18)]),
 (1, [('banana', 2), ('it', 18), ('what', 10)])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.030403747013289406),
   (None, 0.07137715594990957),
   (None, 0.12268959138645896),
   (None, 0.20205849084851002),
   (None, 0.21527155870720083),
   (None, 0.2259445819330831),
   (None, 0.23737185012498374),
   (None, 0.25571291113625794),
   (None, 0.2923689359713647),
   (None, 0.327452525769068),
   (None, 0.32868600350412847),
   (None, 0.3613354382815217),
   (None, 0.4060800536852024),
   (None, 0.40881849399665693),
   (None, 0.4339129908022107),
   (None, 0.4572024835149796),
   (None, 0.4722002000938914),
   (None, 0.4738193979711083),
   (None, 0.48527529237034717),
   (None, 0.4876563060307342),
   (None, 0.4944241216330024)]),
 (1,
  [(None, 0.613592286123381),
   (None, 0.6152963438159947),
   (None, 0.6389357952605998),
   (None, 0.647811577626427),
   (None, 0.6948044419349376),
   (None, 0.7283936124888797),
   (None, 0.7484142431582969),
   (None, 0.8009999394408991),
   (None, 0.9782405179915273)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [ ]:
import numpy as np

data = np.random.rand(30)
print("Сгенерированные числа:", data)

def reader():
    for num in data:
        yield (None, num)

def mapper(_, num):
    yield (None, num)

def reducer(_, nums):
    yield (None, max(nums))

result = list(MapReduce(reader, mapper, reducer))
print('Max: ')
print(result)

Сгенерированные числа: [0.99226213 0.00706443 0.81454458 0.20519865 0.87625575 0.33782037
 0.5875697  0.32709176 0.2549405  0.94311649 0.23457287 0.34080241
 0.31594738 0.11651511 0.16645965 0.01225437 0.38049413 0.12770215
 0.09887593 0.05756027 0.77981603 0.73354478 0.81181197 0.45030652
 0.12081095 0.88993942 0.51182407 0.7372745  0.72713229 0.79606257]
Max: 
[(None, 0.9922621349436724)]


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [ ]:
import numpy as np

data = np.random.rand(30)
print("Сгенерированные числа:", data)

def reader():
    for num in data:
        yield (None, num)

def mapper(_, num):
    yield (None, num)

def reducer(_, nums):
    yield (None, sum(nums) / len(nums))

result = list(MapReduce(reader, mapper, reducer))

print("Среднее значение:", result[0][1])


Сгенерированные числа: [0.8095183  0.73927341 0.2865916  0.18196723 0.66207097 0.94574946
 0.22336667 0.93637405 0.38208611 0.36606334 0.36514698 0.85518452
 0.94308407 0.04603045 0.05794282 0.71964888 0.69161619 0.88632807
 0.18287596 0.65288735 0.06513451 0.37223182 0.03050268 0.91276804
 0.68338709 0.66612318 0.84021123 0.08396496 0.34613599 0.40823042]
Среднее значение: 0.511416545643757


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [ ]:

def groupbykey_sorted(iterable):
    sorted_iterable = sorted(iterable, key=lambda x: x[0])
    grouped_data = []

    current_key, current_group = sorted_iterable[0][0], []
    for key, value in sorted_iterable:
        if key == current_key:
            current_group.append(value)
        else:
            grouped_data.append((current_key, current_group))
            current_key, current_group = key, [value]

    grouped_data.append((current_key, current_group))
    return grouped_data

input_data = [
    ("a", 1), ("b", 2), ("a", 3), ("b", 4), ("c", 5), ("a", 6)
]

result = groupbykey_sorted(input_data)
print(result)


[('a', [1, 3, 6]), ('b', [2, 4]), ('c', [5])]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [ ]:
from itertools import chain

num_reducers = 2
num_mappers = 2

def input_format():
    data = ["apple", "banana", "apple", "orange", "banana", "grape", "grape", "apple"]
    chunk_size = max(1, len(data) // num_mappers)

    def record_reader(chunk):
        for item in chunk:
            yield (item, None)

    for i in range(0, len(data), chunk_size):
        yield record_reader(data[i:i+chunk_size])

def mapper(key, _):
    yield (key, None)

def partitioner(key):
    return hash(key) % num_reducers

def reducer(key, _):
    yield (key, None)

unique_values = MapReduceDistributed(input_format, mapper, reducer, partitioner)
unique_values = [key for (_, partition) in unique_values for (key, _) in partition]

print("Unique values:", unique_values)


8 key-value pairs were sent over a network.
Unique values: ['apple', 'banana', 'grape', 'orange']


#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [ ]:
def predicate(t):
    return (t[0] + t[1]) % 2 == 0

def input_format():
    dataset = [(0, 2), (6, 1), (5, 5)]
    for t in dataset:
        yield (t, None)

def mapper(t, _):
    if predicate(t):
        yield (t, t)

def reducer(_, values):
    yield from values

result = list(MapReduce(input_format, mapper, reducer))
print("Filtered tuples:", result)

Filtered tuples: [(0, 2), (5, 5)]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [ ]:
# Устанавливаем нужные атрибуты для фильтрации
attributes_to_keep = {1, 5, 10}

def MAP(record, _):
    """Оставляем только нужные поля из записи."""
    filtered = {key: value for key, value in record.items() if key in attributes_to_keep}
    sorted_key = tuple(sorted(filtered.items()))  # Сортируем и создаем кортеж для предсказуемости
    yield (sorted_key, sorted_key)  # Возвращаем пару (отфильтрованная запись, отфильтрованная запись)

def REDUCE(_, grouped_values):
    """Удаляем дубликаты, оставляя только один экземпляр."""
    yield grouped_values[0]  # Возвращаем только первый элемент

def RECORDREADER():
    """Генератор, который подает записи данных."""
    yield ({1: "apple", 2: "banana", 5: "grape"}, None)
    yield ({1: "orange", 3: "pear", 10: "kiwi"}, None)
    yield ({5: "melon", 7: "cherry", 10: "mango"}, None)

# Выполнение MapReduce с использованием ранее определенных функций
final_result = list(MapReduce(RECORDREADER, MAP, REDUCE))
# Вывод результата
print("Проекция по атрибутам:", final_result)

Проекция по атрибутам: [((1, 'apple'), (5, 'grape')), ((1, 'orange'), (10, 'kiwi')), ((5, 'melon'), (10, 'mango'))]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [ ]:
def MAP(record, _):
    """Каждый элемент данных превращается в пару (record, record)."""
    yield (record, record)

def REDUCE(record, grouped_records):
    """Удаляет повторяющиеся элементы и возвращает только один экземпляр."""
    yield (record, record)  # Значения уже сгруппированы, возвращаем первый экземпляр

def RECORDREADER():
    """Генератор данных, который объединяет два множества."""
    set_a = [(1, None), (2, None), (3, None)]
    set_b = [(2, None), (4, None), (5, None)]
    return set_a + set_b  # Симуляция объединения множества A с множеством B

# Запуск MapReduce с использованием заданных функций
output = list(MapReduce(RECORDREADER, MAP, REDUCE))

# Вывод результата
print("Объединенный набор данных:", output)

Объединенный набор данных: [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [ ]:
def MAP(element, _):
    """Каждый элемент преобразуется в пару (element, element)."""
    yield (element, element)

def REDUCE(element, grouped_elements):
    """Если элемент встречается дважды, добавляем его в итоговый результат."""
    if len(grouped_elements) == 2:
        yield (element, element)

def RECORDREADER():
    """Генератор данных для пересечения двух множеств: {1, 2, 3, 4} и {2, 3, 5, 6}."""
    set_x = [(1, None), (2, None), (3, None), (4, None)]
    set_y = [(2, None), (3, None), (5, None), (6, None)]
    return set_x + set_y  # Объединение данных для имитации обработки

# Выполнение MapReduce
intersection_result = list(MapReduce(RECORDREADER, MAP, REDUCE))

# Выводим результат пересечения
print("Пересечение двух множеств:", intersection_result)


Пересечение двух множеств: [(2, 2), (3, 3)]


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [ ]:
def MAP(item, source):
    """Помечаем каждый элемент его источником (R или S)."""
    yield (item, source)

def REDUCE(item, sources):
    """Добавляем элемент в результат, если он присутствует только в R."""
    if sources == ["R"]:  # Это означает, что элемент не был в S
        yield (item, item)

def RECORDREADER():
    """Создаем два множества: R и S."""
    set_r = [(1, "R"), (2, "R"), (3, "R"), (4, "R")]  # Множество R
    set_s = [(2, "S"), (3, "S"), (5, "S"), (6, "S")]  # Множество S
    return set_r + set_s  # Объединяем множества для обработки

# Запуск MapReduce
difference_result = list(MapReduce(RECORDREADER, MAP, REDUCE))

# Выводим результат разности R - S
print("Разность множества R и S:", difference_result)


Разность множества R и S: [(1, 1), (4, 4)]


### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [ ]:
def MAP(record, source):
    """Генерирует пары (b, (источник, значение)) для объединения."""
    if source == "R":
        a, b = record
        yield (b, ("R", a))  # Формируем пару для первого множества
    elif source == "S":
        b, c = record
        yield (b, ("S", c))  # Формируем пару для второго множества

def REDUCE(b, grouped_values):
    """Создает комбинации (a, b, c) для подходящих соединений."""
    r_values = [a for src, a in grouped_values if src == "R"]
    s_values = [c for src, c in grouped_values if src == "S"]

    for a in r_values:
        for c in s_values:
            yield (a, b, c)  # Возвращаем комбинированные значения

def RECORDREADER():
    """Генерирует данные для соединения: R(a, b) и S(b, c)."""
    set_r = [("X", 1), ("Y", 2), ("Z", 3)]  # (a, b)
    set_s = [(1, "apple"), (2, "banana"), (3, "cherry")]  # (b, c)
    return [(r, "R") for r in set_r] + [(s, "S") for s in set_s]

# Запуск MapReduce
join_result = list(MapReduce(RECORDREADER, MAP, REDUCE))

# Выводим результат соединения
print("Результат объединения (Natural Join):", join_result)


Результат объединения (Natural Join): [('X', 1, 'apple'), ('Y', 2, 'banana'), ('Z', 3, 'cherry')]


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [ ]:
from collections import defaultdict
from typing import Callable, List, Tuple

def MAP(a, b, c):
    """Формируем пару (a, b) для последующей агрегации."""
    yield (a, b)

def REDUCE(a, values, aggregate_fn: Callable[[List[int]], int]):
    """Применяем агрегирующую функцию и возвращаем результат для ключа a."""
    yield (a, aggregate_fn(values))

def RECORDREADER():
    """Пример данных: (a, b, c)."""
    return [(1, 10, "x"), (2, 20, "y"), (1, 15, "z"), (2, 25, "w"), (3, 30, "v")]

def shuffle_data(mapped_data: List[Tuple[int, int]]) -> dict:
    """Группируем данные по ключу."""
    grouped = defaultdict(list)
    for key, value in mapped_data:
        grouped[key].append(value)
    return grouped

def MapReduce(input_data_func, map_fn, reduce_fn, aggregate_fn):
    """Основная логика MapReduce."""
    records = input_data_func()

    # Этап MAP
    mapped = []
    for record in records:
        mapped.extend(map_fn(*record))  # Распаковываем и применяем функцию map

    # Этап SHUFFLE
    grouped = shuffle_data(mapped)

    # Этап REDUCE
    reduced = []
    for key, values in grouped.items():
        reduced.extend(reduce_fn(key, values, aggregate_fn))

    return reduced

# Пример использования с суммированием
print("Результат агрегации SUM:", MapReduce(RECORDREADER, MAP, REDUCE, sum))

# Пример использования с нахождением максимума
print("Результат агрегации MAX:", MapReduce(RECORDREADER, MAP, REDUCE, max))


Результат агрегации SUM: [(1, 25), (2, 45), (3, 30)]
Результат агрегации MAX: [(1, 15), (2, 25), (3, 30)]


#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [ ]:
from typing import List, Tuple, Dict, NamedTuple
from collections import defaultdict

NUM_REDUCERS = 2  # Количество редукторов
CHUNK_SIZE = 2  # Размер чанка для вектора

# Функция отображения для матрицы
def map_matrix_row(row: NamedTuple) -> Tuple[int, Tuple[str, NamedTuple]]:
    return row.col % NUM_REDUCERS, ('M', row)

# Функция отображения для векторных чанков
def map_vector_chunk(vector_chunk: List[NamedTuple]) -> List[Tuple[int, Tuple[str, NamedTuple]]]:
    return [(el.index % NUM_REDUCERS, ('V', el)) for el in vector_chunk]

# Функция редукции (умножение строк матрицы на соответствующие элементы вектора)
def reduce_product(reducer_id: int, values: List[Tuple[str, NamedTuple]]) -> List[Tuple[int, float]]:
    matrix_rows, vector_elements = [], []
    for tag, record in values:
        if tag == 'M':
            matrix_rows.append(record)
        else:
            vector_elements.append(record)

    return [(row.row, row.value * el.value) for row in matrix_rows for el in vector_elements if row.col == el.index]

# Функция группировки данных по редукторам
def shuffle_data(mapped_data: List[Tuple[int, Tuple[str, NamedTuple]]]) -> Dict[int, List[Tuple[str, NamedTuple]]]:
    grouped = defaultdict(list)
    for reducer_id, pair in mapped_data:
        grouped[reducer_id].append(pair)
    return grouped

# Финальная агрегация (складывает результаты умножения)
def aggregate_results(partial_results: List[Tuple[int, float]]) -> Dict[int, float]:
    result = defaultdict(float)
    for row, value in partial_results:
        result[row] += value
    return result

# Чтение данных матрицы
def read_matrix_data(matrix: List[NamedTuple]) -> List[Tuple[str, NamedTuple]]:
    return [('M', row) for row in matrix]

# Разбиение вектора на чанки
def split_vector(vector: List[NamedTuple]) -> List[List[NamedTuple]]:
    return [vector[i:i + CHUNK_SIZE] for i in range(0, len(vector), CHUNK_SIZE)]

# Основная функция MapReduce
def map_reduce(matrix, vector):
    mapped_matrix = [map_matrix_row(row) for _, row in read_matrix_data(matrix)]
    mapped_vector = [pair for chunk in split_vector(vector) for pair in map_vector_chunk(chunk)]
    grouped = shuffle_data(mapped_matrix + mapped_vector)
    partial_results = [res for rid, vals in grouped.items() for res in reduce_product(rid, vals)]
    return aggregate_results(partial_results)

from collections import namedtuple
MatrixRow = namedtuple('MatrixRow', ['row', 'col', 'value'])
VectorElement = namedtuple('VectorElement', ['index', 'value'])

# Пример данных для умножения матрицы на вектор
matrix = [MatrixRow(0, 0, 1.0), MatrixRow(0, 1, 2.0), MatrixRow(1, 0, 3.0), MatrixRow(1, 1, 4.0)]
vector = [VectorElement(0, 0.5), VectorElement(1, 0.7)]

# Результат умножения
result = map_reduce(matrix, vector)
print("Результат умножения матрицы на вектор:", result)


Результат умножения матрицы на вектор: defaultdict(<class 'float'>, {0: 1.9, 1: 4.3})


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [ ]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))


Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [ ]:
import numpy as np

I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)  # Маленькая матрица в памяти
big_mat = np.random.rand(J, K)  # Большая матрица читается через RECORDREADER

def RECORDREADER():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((j, k), big_mat[j, k])  # (j,k) - ключ, n_{jk} - значение

def MAP(k1, v1):
    (j, k) = k1  # Из ключа (j, k) извлекаем индексы
    w = v1  # n_{jk}

    for i in range(small_mat.shape[0]):  # Пробегаем все строки маленькой матрицы
        v = small_mat[i, j]  # m_{ij}
        yield ((i, k), v * w)  # (i,k) - ключ, произведение - значение

def REDUCE(key, values):
    (i, k) = key
    yield ((i, k), sum(values))  # Суммируем все произведения

result = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(result)

[((0, 0), 0.6154680954357201), ((1, 0), 0.5107440329096617), ((0, 1), 0.7185411052105499), ((1, 1), 0.2838657794350603), ((0, 2), 0.39308123836199116), ((1, 2), 0.2892228922294574), ((0, 3), 0.7663289084274129), ((1, 3), 0.37170007069229666), ((0, 4), 1.0944256380419226), ((1, 4), 0.8698152368129288), ((0, 5), 1.0097957429441613), ((1, 5), 0.9276441919666509), ((0, 6), 0.3886206808503247), ((1, 6), 0.27371446116746506), ((0, 7), 0.2938393037475123), ((1, 7), 0.08897036708877551), ((0, 8), 1.300118844398048), ((1, 8), 0.7660877849332246), ((0, 9), 0.6551235003463405), ((1, 9), 0.3039804819205024), ((0, 10), 0.1722708778165729), ((1, 10), 0.13078894018829135), ((0, 11), 0.16899151972796317), ((1, 11), 0.0394414268455692), ((0, 12), 0.8777564160074899), ((1, 12), 0.7080278578718691), ((0, 13), 0.5916349463046825), ((1, 13), 0.3416750275546511), ((0, 14), 0.44334138421948777), ((1, 14), 0.2994059163376266), ((0, 15), 0.4920429245471688), ((1, 15), 0.2269963803955291), ((0, 16), 0.535578627

Проверьте своё решение

In [ ]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true


True

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [ ]:
import numpy as np  # Импорт библиотеки NumPy для работы с матрицами

# Размерности матриц
I = 2
J = 3
K = 40

# Генерация случайных матриц
np.random.seed(42)  # Фиксируем seed для воспроизводимости результатов
small_mat = np.random.rand(I, J)  # Матрица M размером (I x J)
big_mat = np.random.rand(J, K)  # Матрица N размером (J x K)

# Эталонное решение с использованием стандартного перемножения матриц
reference_solution = np.matmul(small_mat, big_mat)

# Функция, читающая данные обеих матриц
def RECORDREADER():
    # Чтение первой матрицы (M)
    for i in range(small_mat.shape[0]):
        for j in range(small_mat.shape[1]):
            yield ((0, i, j), small_mat[i, j])  # Формат (0, i, j, M[i, j])

    # Чтение второй матрицы (N)
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((1, j, k), big_mat[j, k])  # Формат (1, j, k, N[j, k])

# Функция MAP для соединения матриц по индексу j
def MAP_JOIN(k1, v1):
    mat_type, i_or_j, j_or_k = k1  # Извлекаем номер матрицы и индексы
    value = v1

    if mat_type == 0:  # Если это элемент первой матрицы (M)
        yield (j_or_k, (0, i_or_j, value))  # Группируем по j
    else:  # Если это элемент второй матрицы (N)
        yield (i_or_j, (1, j_or_k, value))  # Группируем по j

# Функция REDUCE для соединения матриц по j
def REDUCE_JOIN(key, values):
    first_matrix = [v for v in values if v[0] == 0]  # Элементы первой матрицы (M)
    second_matrix = [v for v in values if v[0] == 1]  # Элементы второй матрицы (N)

    for m in first_matrix:
        for n in second_matrix:
            yield ((m[1], n[1]), m[2] * n[2])  # Умножение значений mij * njk

# Функция MAP для финальной агрегации (суммирование произведений)
def MAP_MUL(k1, v1):
    yield (k1, v1)  # Оставляем ключ (i, k) без изменений

# Функция REDUCE для суммирования произведений элементов
def REDUCE_MUL(key, values):
    yield (key, sum(values))  # Складываем все элементы для ключа (i, k)

# Генерация объединённых значений через MapReduce
joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)

# Выполнение финального этапа MapReduce для получения результата
solution = MapReduce(lambda: joined, MAP_MUL, REDUCE_MUL)

# Проверка, совпадает ли результат с эталонным значением
print(np.allclose(reference_solution, asmatrix(solution)))  # Должно вывести True

True


Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [ ]:
import numpy as np

NUM_MAPPERS = 2
reducers = 2
ROWS_A = 2
COLS_A = 3
COLS_B = 40  # K = 4 * 10

# Генерация данных
A = np.random.rand(ROWS_A, COLS_A)
B = np.random.rand(COLS_A, COLS_B)

def generate_chunks():
    """Создаёт части матриц для обработки в MAP."""
    chunk_size = int(np.ceil(ROWS_A / NUM_MAPPERS))
    for i_start in range(0, ROWS_A, chunk_size):
        yield process_chunk(range(i_start, min(i_start + chunk_size, ROWS_A)))

def process_chunk(rows):
    """Генерирует пары (координаты, значение) из двух матриц."""
    for i in rows:
        for j in range(COLS_A):
            for k in range(COLS_B):
                yield ((i, j), A[i, j]), ((j, k), B[j, k])

def map_function(pair_A, pair_B):
    """Умножает элементы и группирует по (i, k)."""
    (i, j), val_A = pair_A
    (j, k), val_B = pair_B
    yield (i, k), val_A * val_B

def reduce_function(index, values):
    """Суммирует элементы с одинаковыми индексами."""
    yield index, sum(values)

# Запуск MapReduce
intermediate_results = MapReduceDistributed(generate_chunks, map_function, reduce_function)

# Объединение результатов
final_result = {}
for partition_id, partition in intermediate_results:
    for index, value in partition:
        final_result[index] = final_result.get(index, 0) + value


print("Matrix Multiplication Result:\n", final_result)


240 key-value pairs were sent over a network.
Matrix Multiplication Result:
 {(0, 1): 0.47268518561234973, (0, 2): 1.331585175830705, (0, 4): 1.021351852495352, (0, 5): 0.9131445349756118, (0, 8): 1.0854341486196752, (0, 9): 0.3075827214969634, (0, 11): 1.003832121786526, (0, 12): 0.45686569948009204, (0, 15): 0.8918100486065025, (0, 16): 0.5041984539756577, (0, 18): 0.8286369376706371, (0, 19): 0.9717206052489281, (0, 22): 1.099447290027778, (0, 23): 0.5036027285086736, (0, 25): 1.395355376421955, (0, 26): 0.9624208429352515, (0, 29): 0.5962891831345462, (0, 32): 0.5959378597563265, (0, 33): 0.6976597508277623, (0, 36): 0.04441053975372091, (0, 39): 0.6117816957363831, (1, 0): 0.4197798780682388, (1, 1): 0.31678983211806466, (1, 3): 0.2905523671212979, (1, 4): 0.6256497395186525, (1, 7): 0.7423569849982339, (1, 10): 0.6607515214848413, (1, 11): 0.6015564366326857, (1, 14): 0.8685641215901408, (1, 17): 0.38345055566397046, (1, 18): 0.6792902693535021, (1, 21): 0.6236422180184992, (1, 2

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [ ]:
import numpy as np

# Определение размеров матриц
rows_a = 2
cols_a = 3
cols_b = 4 * 10

# Генерация случайных матриц
matrix_a = np.random.rand(rows_a, cols_a)
matrix_b = np.random.rand(cols_a, cols_b)

# Получение эталонного результата через умножение матриц
expected_result = np.matmul(matrix_a, matrix_b)

# Функция для "разглаживания" вложенных итерируемых объектов
def flatten_iterables(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

# Функция для группировки элементов по ключу
def group_elements_by_key(iterable):
    grouped = {}
    for (key, value) in iterable:
        grouped[key] = grouped.get(key, []) + [value]
    return grouped.items()

# Функция для распределенной группировки элементов по ключу
def distributed_grouping(map_partitions, partition_func):
    global num_reducers
    partitions = [dict() for _ in range(num_reducers)]
    for map_partition in map_partitions:
        for (key, value) in map_partition:
            partition = partitions[partition_func(key)]
            partition[key] = partition.get(key, []) + [value]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

# Функция для определения разделителя (по ключу)
def partition_function(obj):
    global num_reducers
    return hash(obj) % num_reducers

# Функция для выполнения MapReduce на распределенных данных
def distributed_map_reduce(input_format, map_func, reduce_func, partition_func=partition_function, combiner=None):
    map_partitions = map(lambda record_reader: flatten_iterables(map(lambda kv: map_func(*kv), record_reader)), input_format())

    if combiner is not None:
        map_partitions = map(lambda map_partition: flatten_iterables(map(lambda kv: combiner(*kv), group_elements_by_key(map_partition))), map_partitions)

    reduce_partitions = distributed_grouping(map_partitions, partition_func)

    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten_iterables(map(lambda reduce_input_group: reduce_func(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    return reduce_outputs

# Функция для преобразования результата REDUCE в матрицу
def convert_to_matrix(reduce_output):
    reduce_output = list(reduce_output)
    rows = max(i for ((i, k), value) in reduce_output) + 1
    cols = max(k for ((i, k), value) in reduce_output) + 1
    result_matrix = np.empty(shape=(rows, cols))
    for ((i, k), value) in reduce_output:
        result_matrix[i, k] = value
    return result_matrix

# Генератор для вводных данных
def input_data_format():
    data_a = []

    for i in range(matrix_a.shape[0]):
        for j in range(matrix_a.shape[1]):
            data_a.append(((0, i, j), matrix_a[i, j]))  # первая матрица

    global num_maps
    split_size = int(np.ceil(len(data_a) / num_maps))

    for i in range(0, len(data_a), split_size):
        yield data_a[i:i + split_size]

    data_b = []

    for j in range(matrix_b.shape[0]):
        for k in range(matrix_b.shape[1]):
            data_b.append(((1, j, k), matrix_b[j, k]))  # вторая матрица

    split_size = int(np.ceil(len(data_b) / num_maps))

    for i in range(0, len(data_b), split_size):
        yield data_b[i:i + split_size]

# MAP функция для соединения матриц
def map_join_func(key1, value1):
    (matrix_id, i, j) = key1
    weight = value1

    if matrix_id == 0:
        yield (j, (matrix_id, i, weight))
    else:
        yield (i, (matrix_id, j, weight))

# REDUCE функция для соединения матриц
def reduce_join_func(key, values):
    from_first_matrix = [v for v in values if v[0] == 0]
    from_second_matrix = [v for v in values if v[0] == 1]

    for f in from_first_matrix:
        for s in from_second_matrix:
            yield ((f[1], s[1]), f[2] * s[2])

# Генератор для получения соединенных данных
def generate_joined_data():
    for j in joined_data:
        yield j[1]

# MAP функция для передачи значений
def map_multiplication(key1, value1):
    yield (key1, value1)

# REDUCE функция для суммирования значений
def reduce_multiplication(key, values):
    total_value = 0

    for v in values:
        total_value += v
    yield (key, total_value)

num_maps = 3
num_reducers = 2

# Выполнение MapReduce для соединения матриц
partitioned_result = distributed_map_reduce(input_data_format, map_join_func, reduce_join_func, combiner=None)
joined_data = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_result]

# Выполнение MapReduce для умножения значений
multiplication_output = distributed_map_reduce(generate_joined_data, map_multiplication, reduce_multiplication, combiner=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in multiplication_output]

# Формирование окончательного результата
final_solution = []

for p in pre_result:
    for v in p[1]:
        final_solution.append(v)

# Проверка на соответствие с эталонным решением
is_close = np.allclose(expected_result, convert_to_matrix(final_solution))

# Вывод части матрицы
result_matrix = convert_to_matrix(final_solution)
print("Результат умножения матриц (части первой и последней строки):")
print(result_matrix[0])  # Первая строка
print(result_matrix[-1])  # Последняя строка
print("Сравнение с эталонным решением:", is_close)


Результат умножения матриц (части первой и последней строки):
[0.76297669 0.12634115 0.6231596  1.09131914 0.31992627 0.71671965
 1.04442238 1.04414646 0.86452637 0.70794068 0.62148223 0.4417982
 0.91327461 0.85955059 0.85767505 1.10999319 0.71212365 0.75631386
 0.81355796 0.70741363 0.8064103  0.8541745  0.99737019 0.62318393
 0.60657404 0.26118678 0.95438471 0.32988915 0.73488937 0.80918288
 0.41005144 0.59314749 0.1558566  0.27505499 0.72071451 0.45912213
 0.13479107 0.5114723  1.06968427 0.41064228]
[1.23057179 0.20210192 0.73299819 1.37824387 0.70467077 1.18731333
 1.44562747 1.7546595  1.31348039 0.72501421 1.20532402 0.85263036
 1.21179633 1.12066193 0.96498958 1.52308862 1.11644083 1.37792744
 0.84578768 0.9318483  1.19173359 1.07791162 1.2798951  1.30491628
 1.14612965 0.69415626 1.87117636 0.97953737 1.44722403 1.50496154
 0.6886336  0.60916879 0.42037259 0.74293971 0.53723468 0.78460963
 0.16336247 0.55093071 1.81763528 0.79510005]
Сравнение с эталонным решением: True
